# 代码重构

### 在不改变代码外在行为的前提下，对代码做出修改，以改进程序的内部结构。

* <font size=4>
    一个影片出租店的程序，计算每一个顾客的消费金额并打印报表（statement）。操作者告诉程序：顾客租了那些影片、租期多长、程序便根据租赁时间和
  影片类型算出费用。影片分为三级：普通片、儿童片、新片。除了计算费用，还要为常客计算点数：点数会随着【租片种类是否为新片】而有所不同。
</font>
 

In [84]:
class Movie:
    
    childrens=2
    regular=0
    newRelease=1
    
    def __init__(self,title='',priceCode=0):
        self.title=title
        self.priceCode=priceCode
    
   
    

In [85]:
"""
表示顾客租了一部影片

"""
class Rental:
    
    def __init__(self,movie,daysRented):
        self.movie=movie
        self.daysRented=daysRented
        
        
    

In [98]:
class Customer:
    
    
    
    def __init__(self,name,arg,totalAmount,frequentRenterPoints):
        self.name=name
        self.list=arg
        self.totalAmount=totalAmount
        self.frequentRenterPoints=frequentRenterPoints
     
    def addRental(self,arg):
        self.list.add(arg)
        
    def statement(self):
        result="Rental Record for "+self.name+"\n"
        lists=self.list
        for rental in lists:
            thisAmount=0
            if rental.movie.priceCode==Movie.regular:
                thisAmount +=2
                if rental.daysRented > 2:
                    thisAmount +=(rental.daysRented-2)*1.5
            elif rental.movie.priceCode==Movie.newRelease:
                thisAmount +=rental.daysRented*3
            else: 
                thisAmount +=1.5
                if rental.daysRented > 3:
                    thisAmount +=(rental.daysRented -3) *1.5
            """
            add frequent renter points
            """
            self.frequentRenterPoints +=1
            """
            add bonus for a two day new release rental
            """
            if (rental.movie.priceCode==Movie.newRelease) and (rental.daysRented)>1:
                self.frequentRenterPoints+=1
            
            result+="\t"+rental.movie.title+"\t"+str(thisAmount)+"\n"
            self.totalAmount+=thisAmount
            
        """
           add footer lines
        """
        result+="Amount owed is "+str(self.totalAmount)+"\n"
        result+="You earned "+str(self.frequentRenterPoints)+" frequent renter points"
        
        print  (result)
        return result
        
    

In [146]:
class Test:
     
    def prt(self):
        
        movie1=Movie('绿野仙踪',2)
        movie2=Movie('美女与野兽',0)
        movie3=Movie('仙履奇缘',1)
        
        rental1=Rental(movie1,2)
        rental2=Rental(movie2,4)
        rental3=Rental(movie3,5)
        
        list1=[rental1,rental2,rental3]
        
        customer=Customer('小罗',list1,0,0)
        
        result=customer.statement()

t=Test()
t.prt()
        
        
        
        
        
        

Rental Record for 小罗
	绿野仙踪	1.5
	美女与野兽	5.0
	仙履奇缘	15
Amount owed is 21.5
You earned 4 frequent renter points


### 对这段程序感官如何？
* <font size=4>
    首先最直观的感受就是在长长的statement里做的事情太多了，这些事原本应该由其他类完成。这是典型的面向过程编程。例如，如果客户想打印出更漂亮的账单。用html标签将样式加进去。它能做的就是重新写个htmlstatement。新方法需要大量重复statement的行为。又如果计费标准改变了呢？你必须修改statement的同时又要修改htmlstatement。
</font>


* <font size=4 color=red>如果你发现自己需要为程序添加一个新特性，而代码结构让你无法方便的达到目的，那么你就需要重构。</font>
    
### 开始重构

* <font size=4>重构前，你需要有一套可靠的测试机制，这些测试必须有自我检验能力：例如单元测试里的断言。详尽的测试案例在重构中十分重要</font>


*  <font size=4>分解并重组statement()：我们需要把这长长的函数切开，并把较小块的diamante移到合适的类。降低代码重复量，从而使新的打印详情的函数更容易撰写</font>


* <font size=4>任何一个傻瓜都能写出计算机可以理解的代码，唯有写出人类容易理解的代码才是优秀的程序员：合理的变量名、清晰的代码结构。</font>


* <font size=4> 我们发现这个函数里计算价格的方法使用了Rental类信息，却并没有使用Customer类信息，绝大多数情况下，函数应该放在它所使用的数据的所属对象内，那么我们可以把计算价格这段代码迁移到Rental类中 </font>


* <font size=4>去掉临时变量：临时变量只在自己所属的函数中有效，它会助长冗长且复杂的函数。这里的两个临时变量totalAmount和frequentRentalPoints用查询函数来取代。由于任何函数都可以调用上述查询函数，从代码复用角度，它能促成较干净的设计而减少冗长复杂的函数。</font>


* <font size=4>if语句太多不例如代码的扩展，遇到这类情况请使用多态来解决。计算价格是根据影片的类型关联，那么这块代码根据上面讲述的原则放到Movie类中更好。</font>

* <font size=4>对设计模式的思考：如果建立Movie的三个子类，各自有自己的计算方法。类图如下：</font>


<img src="https://github.com/wanjun6612/wanjundecangku/blob/master/markDownPic/TIM%E5%9B%BE%E7%89%8720180213004355.jpg?raw=true">

* <font size=4>这样利用多态就取代了过多的条件语句，后续扩展新类型只用添加新类，而原来的主干代码不用修改。从而更符合开闭原则、面向接口编程等要求。但这里有一个小问题。一部影片可以在自己的生命周期里修改自己的分类，一个对象却不能再生命周期内修改自己所属的类。但是这里有一个状态模式，运用它之后，类图如下：</font>

<img src="https://github.com/wanjun6612/wanjundecangku/blob/master/markDownPic/TIM%E5%9B%BE%E7%89%8720180213004743.jpg?raw=true">

* <font size=4>状态模式与策略模式的区别：策略模式一般用于单个算法，而状态模式的每个状态子类中需要包含所有原来的语境类（Context）中的所有方法的具体实现，可以通过set方法来切换需实现的策略，即切换上下文（Context）类。而策略模式不存在切换状态的操作，它是直接依赖注入到Context类的参数进行策略选择，类图关系如下：</font>

<img src="https://github.com/wanjun6612/wanjundecangku/blob/master/markDownPic/TIM%E5%9B%BE%E7%89%8720180215235412.jpg?raw=true">




In [271]:
class Movie2:
    
    childrens=2
    regular=0
    newRelease=1
    
    def __init__(self,title='',priceCode=0):
        self.title=title
        self.priceCode=priceCode
        
    
    def setPrice(self):
        if self.priceCode==Movie2.regular:
            self.price=RegularPrice()
            
        elif self.priceCode==Movie2.childrens:
             self.price=ChildrensPrice()
        else:
            self.price=NewReleasePrice()

"""
价格基类
"""   

class Price:
    
    def getCharge(self,days=0):
        pass

    def getFrequentRenterPoints(self,days=0):
        pass

"""
价格状态类
"""   
class RegularPrice:
    
    def getCharge(self,days=0):
        result=2
        if days>2:
            result+=(days-2)*1.5
        return result
    
    def getFrequentRenterPoints(self,days=0):
        return 1
    
    
class ChildrensPrice:
    
    def getCharge(self,days=0):
        result=1.5
        if days > 3:
            result+=(days-3)*1.5       
        return result
    
    def getFrequentRenterPoints(self,days=0):
        return 1

class NewReleasePrice:
    
    def getCharge(self,days=0):
        result=days*3      
        return result
    
    def getFrequentRenterPoints(self,days=0):
        if days > 1:
            result=2
        else:
            result=0
        return result
    
class Rental2:

    def __init__(self,movie2,days=0):
        self.movie2=movie2
        self.days=days
     
    def getCharge(self):
        result=self.movie2.pirce.getCharge()
        return result
    
    def getFrequentRenterPoints(self):
        result=self.movie2.pirce.getFrequentRenterPoints()
        return result


"""这里有个奇怪的地方，如果直接打印self.totalAmount  结果是错误的 ，只有把他们保存在临时变量  还有原方法必须return值 否则也获取不了正确答案
"""   
class   Customer2:

    def __init__(self,name,arg,totalAmount,frequentRenterPoints):
        
        self.name=name
        self.list=arg
        self.totalAmount=totalAmount
        self.frequentRenterPoints=frequentRenterPoints
        
    def getTotalCharge(self):
        self.totalAmount=0;
        lists=self.list
        for rental in lists:
            rental.movie2.setPrice()
            self.totalAmount+=rental.movie2.price.getCharge(rental.days)
        return self.totalAmount
       
    def getTotalFrequentRenterPoints(self):
        self.totalAmount=0;
        for rental in self.list:
            rental.movie2.setPrice()
            self.frequentRenterPoints+=rental.movie2.price.getFrequentRenterPoints(rental.days)
        return self.frequentRenterPoints
   
    def statement(self):
      
        result="Rental Record for "+self.name+"\n"       
       
        for rental in self.list:
            rental.movie2.setPrice()
            result+="\t"+rental.movie2.title+"\t"+str(rental.movie2.price.getCharge(rental.days))+"\n"
        
        totalAmount=self.getTotalCharge()
        frequentRenterPoints=self.getTotalFrequentRenterPoints()
        
        result+="Amount owed is "+str(totalAmount)+"\n"
        result+="You earned "+str(frequentRenterPoints)+" frequent renter points"
        print  (result)
        return result

 
 

In [272]:
   if __name__=='__main__':
        
        movie1=Movie2('绿野仙踪',2)
        movie2=Movie2('美女与野兽',0)
        movie3=Movie2('仙履奇缘',1)
       
        
        rental1=Rental2(movie1,2)
        rental2=Rental2(movie2,4)
        rental3=Rental2(movie3,5)
        
        list1=[rental1,rental2,rental3]
        
        customer=Customer2('小罗',list1,0,0)
        
        result=customer.statement() 

Rental Record for 小罗
	绿野仙踪	1.5
	美女与野兽	5.0
	仙履奇缘	15
Amount owed is 21.5
You earned 4 frequent renter points


### 重构后的类图

<img src="https://github.com/wanjun6612/wanjundecangku/blob/master/markDownPic/TIM%E5%9B%BE%E7%89%8720180215235412.jpg?raw=true">